In [60]:
import numpy as np
from scipy.special import gamma
import sys
import copy

In [61]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return (idx, array[idx])

In [62]:
casename = 'golovin_sourcesink'

# PARAMETERS
Tmax = 14400
nbin = 32
dt = 100. #s
scal = 1 # scalaing factor for grid
emin = 7.091336e-10 # mg; minimum mass for grid
rhow = 1e-9 # mg/um^3

# precip
v_precip = 1000

nt = int(Tmax/dt)

printtimes = np.asarray([0, Tmax/4/dt, Tmax/dt],dtype='int')

B = 1500*1e-12 #1/sec
inject_rate = 1 #1/cm3/sec

# injection condition
k_in = 3
theta_in = 100 #um^3

# initial condition
k1 = 3
theta = 100 #um^3
N0 = 0 # cm^-3

In [63]:
# function for initial condition
def gamma_init(x, N, k, theta):
    try:
        return N*(x/theta)**k/gamma(k)*np.exp(-x/theta)*3*x
    except:
        return 0.0

# functions for kernel
def golovin_kernel(x1,x2,B):
    return B*(x1+x2)

def const_kernel(x1,x2,A):
    return A

def geometric_kernel(x1,x2,C):
    r1 = (x1/np.pi/4*3)**(1/3)
    r2 = (x2/np.pi/4*3)**(1/3)
    a1 = 4 * np.pi * r1**2
    a2 = 4 * np.pi * r2**2
    return C * (r1+r2)**2 * np.abs(a1 - a2)

In [64]:
def courant(vgrid,dlnr):
    nbin = len(vgrid)
    c = np.zeros((nbin,nbin))
    ima = np.zeros((nbin,nbin))
    for i in range(nbin):
        for j in range(i, nbin):
            v0 = vgrid[i] + vgrid[j]
            for k in range(j, nbin):
                if vgrid[k] >= v0 and vgrid[k-1] <= v0:
                    if c[i,j] < 1-1e-8:
                        kk = k-1
                        c[i,j]=np.log(v0/vgrid[kk])/(3*dlnr)
                    else:
                        kk=k
                        c[i,j] = 0.0
                    ima[i,j]=min(nbin-1,kk)
                    break
            c[j,i] = c[i,j]
            ima[j,i] = ima[i,j]
    return (c,ima)


In [65]:
def lin_interp(cck):
    n = len(cck[0])
    ck = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            jm = max(j-1,0)
            im = max(i-1,0)
            jp = min(j+1,n-1)
            ip = min(i+1,n-1)
            ck[i,j] = 0.125*(cck[i,jm]+cck[im,j]+cck[ip,j]+cck[i,jp])+0.5*cck[i,j]
            if i==j:
                ck[i,j]=0.5*ck[i,j]
    return ck

In [66]:
#def g_init(x):
#    return gamma_init(x,N0,k1,theta) + gamma_init(x,N2,k2,theta2)
def g_init(x):
    return gamma_init(x,N0,k1,theta)

def g_inject(x):
    return gamma_init(x,1.0,k_in,theta_in)

# kernel
def kernel(x,y):
    return golovin_kernel(x,y,B)
    #return const_kernel(x,y,A)
    #return geometric_kernel(x,y,C)

In [67]:
# define grid
vmin = emin / rhow
dlnr = np.log(2)/3/scal
gridax = 2**(1/scal)

vgrid = np.zeros(nbin)

vgrid[0] = vmin*0.5*(gridax+1)
for i in range(nbin-1):
    vgrid[i+1] = vgrid[i]*gridax
rgrid = (vgrid/4/np.pi*3)**(1/3)

bin_precip = find_nearest(vgrid,v_precip)[0]+1

# define initial condition
gin = np.zeros(nbin)
for i in range(nbin):
    gin[i] = g_init(vgrid[i])

# define the kernel
cck = np.zeros((nbin,nbin))
for i in range(nbin):
    for j in range(i,nbin):
        cck[i,j] = kernel(vgrid[i],vgrid[j])
        cck[j,i] = cck[i,j]
ck = lin_interp(cck)
ck = ck*dt*dlnr

# define the injection rate
ginject = np.zeros(nbin)
for i in range(nbin):
    ginject[i] = g_inject(vgrid[i])*inject_rate*dt#*dlnr

# compute the courant numbers
(c,ima) = courant(vgrid,dlnr)

In [68]:
def coad_sourcesink(vgrid, gin, ima, ck, c, ginject, vmax, gmin=1e-20):
    #np.seterr(all='raise')
    g = copy.deepcopy(gin)
    nbin=len(vgrid)
    i0=0
    i1=nbin-1
    # find upper and lower integration limit
    for i in range(nbin):
        i0 = i
        if g[i] >= gmin:
            break
    for i in range(0,nbin-1,-1):
        i1 = i
        if g[i] >= gmin:
            break

    for i in range(i0,i1):
        for j in range(i,i1):
            k = int(ima[i,j])
            kp = k+1
            try:
                v0 = ck[i,j]*g[i]*g[j]
            except:
                v0 = 0.0
            v0 = min(v0, g[i]*vgrid[j])
            if j != k:
                v0 = min(v0,g[j]*vgrid[i])
            try:
                gsi=v0/vgrid[j]
            except:
                gsi = 0.0
            try:
                gsj=v0/vgrid[i]
            except:
                gsj = 0.0
            gsk=gsi+gsj
            g[i]=max(g[i]-gsi,0)
            g[j]=max(g[j]-gsj,0)
            gk=g[k]+gsk
            if gk > gmin and g[kp] > gmin:
                try:
                    x1 = np.log(g[kp]/gk)
                except:
                    print(i,j,k,kp,g[kp],gk)
                flux = gsk/x1*(np.exp(0.5*x1)-np.exp(x1*(0.5-c[i,j])))
                flux = min(min(flux,gk),gsk)
                
                g[k]=gk-flux
                g[kp]+=flux
    for i in range(nbin):
        g[i]=g[i]+ginject[i]
        if vgrid[i] > vmax:
            g[i] = 0

    return g

In [69]:
f = open('bott/'+casename+'.txt', 'w')
f.close()
f = open('bott/'+casename+'.txt', 'a')
f.write(str(casename)+'\n')
# f.write('Tmax, nbin, dt, scal, emin, rhow, B, k1, theta, N0 \n')
# f.write(str([Tmax, nbin, dt, scal, emin, rhow, B, k1, theta, N0]))
f.write('Tmax, nbin, dt, scal, emin, rhow, B, k1, theta, N0 \n')
f.write(str([Tmax, nbin, dt, scal, emin, rhow, B, k1, theta, N0]))
f.write('\n')
f.write('\n')
f.write('v_grid = '+np.array2string(vgrid, separator=', ')+'\n')
f.write('r_grid = '+np.array2string((vgrid/4*3/np.pi)**(1/3), separator=', ')+'\n')
f.write('\n')
f.close()


In [70]:
f = open('bott/'+casename+'.txt', 'a')
bin_mom = np.zeros((3,nt+1))
mprecip = np.zeros(nt+1)

gprev = gin.copy()
for ij in range(nt):
    # calculate the moments, etc.
    bin_mom[0,ij] = np.sum(gprev/vgrid*dlnr)
    bin_mom[1,ij] = np.sum(gprev*dlnr)
    bin_mom[2,ij] = np.sum(gprev*vgrid*dlnr)
    mprecip[ij] = np.sum(gprev[bin_precip:]*dlnr)
    if ij in printtimes:
        print(gprev)
        f.write('gr'+str(ij)+' = '+np.array2string(gprev, separator=', ')+'\n')
    # perform the collision step
    gnext = coad_sourcesink(vgrid, gprev, ima, ck, c, ginject, v_precip)
    gprev = gnext.copy()
bin_mom[0,nt] = np.sum(gprev/vgrid*dlnr)
bin_mom[1,nt] = np.sum(gprev*dlnr)
bin_mom[2,nt] = np.sum(gprev*vgrid*dlnr)
mprecip[nt] = np.sum(gprev[bin_precip:]*dlnr)
if ij+1 in printtimes:
    print(gprev)
    f.write('gr'+str(ij+1)+' = '+np.array2string(gprev, separator=', ')+'\n')

mom_times = np.linspace(0.0,Tmax,nt+1)
f.write('t_bott = '+np.array2string(mom_times, separator=', ')+'\n')
f.write('M0_bott = '+np.array2string(bin_mom[0,:], separator=', ')+'\n')
f.write('M1_bott = '+np.array2string(bin_mom[1,:], separator=', ')+'\n')
f.write('M2_bott = '+np.array2string(bin_mom[2,:], separator=', ')+'\n')
f.write('mprecip_bin = '+np.array2string(mprecip[:], separator=', ')+'\n')
f.close()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[3.17294652e-03 5.01160973e-02 7.81208313e-01 1.18642933e+01
 1.71054784e+02 2.22357562e+03 2.35414536e+04 1.67885019e+05
 6.34585981e+05 6.48217578e+05 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[3.52340133e-03 5.55870949e-02 8.64388440e-01 1.30655750e+01
 1.86688681e+02 2.38775519e+03 2.46249375e+04 1.70121182e+05
 6.37652546e+05 6.48012599e+05 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 